In [ ]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.nn import functional as F

import torchmetrics

import onnx
import onnxruntime.quantization
import onnxruntime
from onnxruntime.quantization import quantize_qat, quantize_static, QuantType

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# WANDB LIBRARY
### IMPORT WANDB AND LOGIN
Note you may have to login using your API key


In [ ]:
import wandb
%env "WANDB_NOTEBOOK_NAME" "demo_wine_wandb_test"
wandb.login()

### Import Dataset

In [ ]:
df = pd.read_csv("./data/wine_data.csv")

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.describe()

# DATA WRANGLING
### Check for Nulls and Duplicates

In [ ]:
df.isna().sum()

In [ ]:
df.duplicated().sum()

# MACHINE LEARNING
### ML PREP

In [ ]:
# Encode target labels with value between 0 and n_classes-1.
# Import Metrics for use with evaluation

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_curve, roc_auc_score, confusion_matrix


In [ ]:
le = LabelEncoder()
df['Class'] = le.fit_transform(df['Class'])
df.sample(10)

In [ ]:
df['Class'].unique()

### SEPARATE FEATURES AND TARGET

In [ ]:
# set the feature variables

df_features = df.drop('Class', axis=1)
df_features.head()

In [ ]:
# Set the target variable

df_target = df[['Class']]
df_target.head()

In [ ]:
# Split dataset in train and test with a ratio of 70-30

from sklearn.model_selection import train_test_split

In [ ]:
X_train, x_test, Y_train, y_test = train_test_split(df_features, 
                                                    df_target,
                                                    test_size=0.3,
                                                     random_state=42)

In [ ]:
X_train.shape, x_test.shape,

In [ ]:
Y_train.shape, y_test.shape

### Convert data to Tensors for Pytorch

In [ ]:
Xtrain = torch.from_numpy(X_train.values).float()
Xtest = torch.from_numpy(x_test.values).float()
print(Xtrain.shape, Xtest.shape)

In [ ]:
print(Xtrain.dtype, Xtest.dtype)

We have successfully converted our  X_data into torch tensors of float32 datatype

In [ ]:
# Reshape tensor to 1D

Ytrain = torch.from_numpy(Y_train.values).view(1,-1)[0]
Ytest = torch.from_numpy(y_test.values).view(1, -1)[0]
print(Ytrain.shape, Ytest.shape)
print(Ytrain.dtype, Ytest.dtype)

We use the **view()** to reshape the tensor.<br>
The loss function doesn't support multi-target and therefore, we should use a 1D Tensor of 1 row containing the labels.<br>
We have successfully converted our y_data

In [ ]:
print(Ytrain.dtype, Ytest.dtype)

## PyTorch
### We create a classifier and define our neural network for our model

### Hyperparameters

In [ ]:
input_size = 13
output_size = 3
hidden_size = 100

In [ ]:
config = dict(
                dataset = "wine dataset",
                architecture = 'Linear', 
                learning_rate = 0.01,
                loss = nn.NLLLoss(),
                optimizer = "adam",
)


In [ ]:
for k,v in config.items():
    print(k, v)

### Define the neural network


class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(config.get("input_size"), config.get("hidden_size"))
        self.fc2 = nn.Linear(config.get("input_size"), config.get("hidden_size"))
        self.fc3 = nn.Linear(config.get("input_size"), config.get("output_size"))

    def forward(self, X):
        X = torch.sigmoid((self.fc1(X)))
        X = torch.sigmoid(self.fc2(X))
        X = self.fc3(X)

        return F.log_softmax(X, dim=-1)

In [ ]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, X):
        X = torch.sigmoid((self.fc1(X)))
        X = torch.sigmoid(self.fc2(X))
        X = self.fc3(X)

        return F.log_softmax(X, dim=-1)

In [ ]:
# instantiate model
model = Net()
# move model to gpu
#model.to(device)
# preview our model
model

### Define Optimizer and Loss Function

In [ ]:
import torch.optim as optim

In [ ]:
#optimizer = optim.Adam(model.parameters(), lr=config.get("learning_rate"))
loss_fn = config.get("loss")

In [ ]:
# Define the optimizer

if config.get("optimizer")=='sgd':
    optimizer = optim.SGD(model.parameters(),lr=config.get("learning_rate"), momentum=0.9)
elif config.get("optimizer")=='adam':
    optimizer = optim.Adam(model.parameters(),lr=config.get("learning_rate"))

In [ ]:
# Define the Loss

if config.get("loss") == "NLLLoss":
    loss_fn = nn.NLLLoss()
elif config.get("loss") == "CrossEntropyLoss":
    loss_fn = nn.CrossEntropyLoss()

# TRAIN AND LOG THE MODEL
### Train the Model for 1000 Epochs
### Log the model Parameters
### Export Model, Import and Set to Eval
### Log Metrics on Model.Eval
### Convert and Export to ONNX

In [ ]:
# TRAIN THE MODEL
#acc = torchmetrics.Accuracy()

epochs = 1000
with wandb.init(project="demo_wandb_test", config = config):
    wandb.watch(model, criterion=None, log="gradients", log_freq=10)
    
    for epoch in range(epochs):
        # zero the gradients
        optimizer.zero_grad()
        # train the model
        Ypred = model(Xtrain)
        # compute the accuract
        acc = torchmetrics.functional.accuracy(Ypred, Ytrain)
        # compute the loss
        loss = loss_fn(Ypred, Ytrain)
        # update the model weights
        loss.backward()
        # optimize the learning parameters and step forward
        optimizer.step()
        # log the metrics
        wandb.log({'Epoch': epoch, "Loss": loss.item(), "Accuracy": acc})


    # SAVE MODEL STATE DICT TO DISK

    wandb.save(torch.save(model.state_dict(), "./models/home_state_dict.pt"))

    # LOAD MODEL FROM DISK and EVALUATE

    new_model =  Net()
    new_model.load_state_dict(torch.load("./models/home_state_dict.pt"))
    new_model.eval()

    # SET THE PREDICTIONS

    predict = new_model(Xtest)
    _, predict_y = torch.max(predict, 1)
    ground = Ypred.detach().numpy()

    # VISUALIZE CONFUSION MATRIX

    wandb.sklearn.plot_confusion_matrix(Ytest, predict_y, labels = [0,1,2])
    #wandb.plot.roc_curve(Ytest, ground, labels=[0,1,2])
    #wandb.plot.roc_curve(predict_y, Ytest)
    # Print Metrics

    wandb.log({"accuracy_score" : accuracy_score(Ytest, predict_y),
    "precision_score" : precision_score(Ytest, predict_y, average='weighted'),
    "recall_score": recall_score(Ytest, predict_y, average="weighted"),
    #"roc_curve": roc_curve(Ytest, predict_y,),
    "roc_auc_score": roc_auc_score(Ytest, predict_y, average="macro",  multi_class='ovr'), 
    
    })
    
    torch.onnx.export(model = model,args =  (Xtrain), f = "./models/home_state_test.onnx", input_names=['input'], output_names = ['output'],
    verbose=True, do_constant_folding=True, opset_version=11)
wandb.finish()


# OPTIMIZE NETWORK WITH SWEEPS
### WANDB SWEEPS

In [ ]:
# Replace agent with own agent line generated from project
!wandb agent markgich/demo_wandb_test/izsvluxh

In [ ]:
sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'loss',
      'goal': 'minimize'   
    },
    'parameters': {
        'epochs': {
            'values': [100, 500, 1000]
        },
        
        'learning_rate': {
            'values': [1e-2, 1e-3, 1e-4, 3e-4, 3e-5, 1e-5]
        },
        'fc_layer_size':{
            'values':[128,256,512]
        },
        'optimizer': {
            'values': ['adam', 'sgd']
        },
    }
}

# ADD HYPERPARAMETER TUNING SWEEPS

In [ ]:
# DEMO OF IT